In [1]:
# Reference: https://www.kaggle.com/kcostya/lstm-models-for-multi-step-time-series-forecast
# https://stackoverflow.com/questions/52778922/stock-prediction-gru-model-predicting-same-given-values-instead-of-future-stoc

# 0.0. IMPORTS
---

In [6]:
# load and clean-up data
import numpy  as np
import pandas as pd

import math

from sklearn.metrics import mean_squared_error
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, RepeatVector, TimeDistributed, ConvLSTM2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D